In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro
import json
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
import tensorflow as tf
from tensorflow import keras
from keras import layers, models, optimizers
from sklearn.model_selection import KFold
import optuna 
import winsound
tf.random.set_seed(47)
np.random.seed(47)
# import keras_tuner as kt

X = pd.read_csv("C:/Users/Administrator/Desktop/PFE/data/new/miscplants_Xp.csv", sep=';')
Y = pd.read_csv("C:/Users/Administrator/Desktop/PFE/data/new/miscplants_Y.csv", sep=';')
M = pd.read_csv("C:/Users/Administrator/Desktop/PFE/data/new/miscplants_M.csv", sep=';', na_values ='missing')

In [2]:
def split_data(Var):

    if Var not in Y.columns or Var not in M.columns:
        raise ValueError(f"Errer Erreur Erreur ! ! !")

    mask = M[Var]

    # Split X and Y based on M.csv values
    X_cal = X[mask == 'cal']
    Y_cal = Y.loc[X_cal.index, Var]

    X_val = X[mask == 'val']
    Y_val = Y.loc[X_val.index, Var]

    X_test = X[mask == 'test']
    Y_test = Y.loc[X_test.index, Var]

    return (X_cal, Y_cal), (X_val, Y_val), (X_test, Y_test)

In [3]:
Var = "adl"

In [4]:
# Data Splitting
(X_cal, Y_cal), (X_val, Y_val), (X_test, Y_test) = split_data(Var)
Y_train = pd.concat([Y_cal, Y_val])
X_train = pd.concat([X_cal, X_val])

# Convert to NumPy arrays
# X_cal = X_cal.to_numpy()
# X_val = X_val.to_numpy()
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

num_features = 700  # Spectral features

# Standardization (using X_cal statistics)
# mean_cal, std_cal = X_cal.mean(), X_cal.std()
# X_cal_N = (X_cal - mean_cal) / std_cal
# X_val_N = (X_val - mean_cal) / std_cal
# Standardization (using X_train statistics)
mean_train, std_train = X_train.mean(axis=0), X_train.std(axis=0)
# mean_train, std_train = X_train.mean(), X_train.std()
X_train_N = (X_train - mean_train) / std_train
X_test_N = (X_test - mean_train) / std_train

# Reshape for 1D CNN (batch_size, sequence_length, channels) (taille du batch,longueur de la séquence,canaux)
# X_cal_f = X_cal_N[..., np.newaxis]  # Shape: (samples, features, 1)
# X_val_f = X_val_N[..., np.newaxis]
X_train_f = X_train_N[..., np.newaxis]
X_test_f = X_test_N[..., np.newaxis]

# X_cal_f.shape, X_val_f.shape, X_train_f.shape, X_test_f.shape
# Y_cal.shape, Y_val.shape
print(f"Y: {Y_train.shape}, {Y_test.shape}")
print(f"X: {X_train.shape}, {X_test.shape}")

Y: (1067,), (356,)
X: (1067, 700), (356, 700)


<!-- # **Modelling** -->

In [5]:
# Check GPU availability
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs Available:", gpus)
else:
    print("No GPU detected; please configure CUDA.")

GPUs Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


<!-- ###  CNN-R_v1E -->

# CNN-R_v1E

In [7]:
def build_model(trial):
    # filter_size = trial.suggest_categorical("filter_size", [50, 100, 300, 700])
    # padding_type = 'valid' if filter_size == 700 else 'same'
    filter_size = 700
    padding_type = 'valid'
    num_filters = trial.suggest_int("num_filters", 1, 32)

    # Dense layers
    num_dense_layers = trial.suggest_int("num_dense_layers", 1, 3)
    dense_units = [trial.suggest_int(f"dense_{i}_units", 8, 128, step=4) for i in range(num_dense_layers)]

    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.6, step=0.005) 
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 0.03, log=True) # log=True for logarithmic scale
    l2_reg = trial.suggest_float("l2_reg", 0.0, 0.1, step=5e-4) 
    batch_size = trial.suggest_int("batch_size", 32, 256, step=16)
    initializer = tf.keras.initializers.he_normal(seed=123)

    inputs = tf.keras.Input(shape=(700,))
    x = tf.keras.layers.Reshape((700, 1))(inputs)
    x = tf.keras.layers.Conv1D(filters = num_filters, kernel_size=filter_size,
                               padding = padding_type,
                               kernel_initializer = initializer,
                               kernel_regularizer = tf.keras.regularizers.l2(l2_reg))(x)
    # print("Conv1D output shape:", x.shape) 
    x = tf.keras.layers.Activation('elu')(x)
    x = tf.keras.layers.Flatten()(x)

    for i, units in enumerate(dense_units):
        x = tf.keras.layers.Dense(units, kernel_initializer=initializer,
                                  kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x)
        x = tf.keras.layers.Activation('elu')(x)
        if num_dense_layers > 1:
            x = tf.keras.layers.Dropout(rate=dropout_rate)(x)

    output = tf.keras.layers.Dense(1, activation="linear", kernel_initializer=initializer)(x)
    model = tf.keras.Model(inputs=inputs, outputs=output)

    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss="mse",
                  metrics=[tf.keras.metrics.RootMeanSquaredError()]) 
    
    return model, batch_size
Modd = "CNN-R_v1E"

In [ ]:
# #old
# def objective(trial):
#     model, batch_size = build_model(trial)
    
#     # Fixed Cal/Val strategy
#     es_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=5e-3, patience=10, restore_best_weights=True)
#     reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10)

#     history = model.fit(X_cal_f, Y_cal,
#                   validation_data=(X_val_f, Y_val),
#                   epochs=300,
#                   batch_size=batch_size,
#                   callbacks=[es_callback, reduce_lr],
#                   verbose=0)
#         # Plot losses for the first trial
#     if trial.number == 1 or trial.number == 26 or trial.number == 45:
#         plt.figure(figsize=(10, 5))
#         plt.plot(history.history['loss'], label='Training Loss')
#         plt.plot(history.history['val_loss'], label='Validation Loss')
#         plt.title(f'Training & Validation Loss - Trial {trial.number}')
#         plt.xlabel('Epoch')
#         plt.ylabel('MSE Loss')
#         plt.legend()
#         plt.grid(True)
#         plt.show()


#     val_score = model.evaluate(X_val_f, Y_val, verbose=1)
#     return val_score[1]  # RMSE

In [8]:
def objective(trial):
    epochs_list = []
    es_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=5e-2, patience=20, restore_best_weights=True)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.4, patience=10)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation
    rmse_scores = []
    history_dicts = []  # to collect all fold histories

    for train_idx, val_idx in kf.split(X_train_f):  
        X_train_cv, X_val_cv = X_train_f[train_idx], X_train_f[val_idx]
        Y_train_cv, Y_val_cv = Y_train.iloc[train_idx], Y_train.iloc[val_idx]

        model, batch_size = build_model(trial)  # rebuild the model for each fold
        history = model.fit(X_train_cv, Y_train_cv,
                            validation_data=(X_val_cv, Y_val_cv),
                            epochs=300,
                            batch_size=batch_size,
                            callbacks=[es_callback, reduce_lr],
                            verbose=0)
        epochs_list.append(len(history.history['loss']))
        history_dicts.append(history.history) # store history for each fold


        val_loss, val_rmse = model.evaluate(X_val_cv, Y_val_cv, verbose=0)
        rmse_scores.append(val_rmse)
        avg_epochs = int(np.mean(epochs_list))
        trial.set_user_attr("avg_epochs", avg_epochs)
        trial.set_user_attr("epochs_list", epochs_list)
        trial.set_user_attr("fold_histories", history_dicts)

    return np.mean(rmse_scores)

In [ ]:
# Start the Optuna study
ii = datetime.now()
iii = str(ii.strftime("%Y-%m-%d_%Hh%M"))
print(" **********************************    Start time:", iii,"   ***********************************","\n")
# Set up the Optuna study 
study = optuna.create_study(
    direction="minimize",
    study_name="/optuna/cnn_hpo",
    # storage=f"sqlite:///optuna/cnn_study__{iii}.db",
    load_if_exists=True
)
study.optimize(objective, n_trials=100)

# Print the best trial
print("Best trial :")
print(f"  RMSE on Val: {study.best_value:.4f}")
print("  Best params:")
for k, v in study.best_trial.params.items():
    print(f"    {k}: {v}")

for i in range(5): winsound.Beep(500, 500)

In [ ]:
# Epochs and plots n stuff
print("Average epochs:", study.best_trial.user_attrs["avg_epochs"])
print("Epochs per fold:", study.best_trial.user_attrs["epochs_list"])

# Pick trial indices
start_idx = 0
middle_idx = len(study.trials) // 2
end_idx = len(study.trials)-1
chosen_idxs = [start_idx, middle_idx, end_idx]

for idx in chosen_idxs:
    trial = study.trials[idx]
    histories = trial.user_attrs.get("fold_histories", [])

    for fold_i, history in enumerate(histories):
        fig, ax1 = plt.subplots(figsize=(10, 4))

        # Plot Losses on left axis
        ax1.plot(history["loss"], label="Training Loss", color='tab:blue')
        ax1.plot(history["val_loss"], label="Validation Loss", color='tab:orange')
        ax1.set_xlabel("Epoch")
        ax1.set_ylabel("Loss", color='black')
        ax1.tick_params(axis='y', labelcolor='black')
        ax1.grid(True)

        # Plot Learning Rate on right axis if available
        if "lr" in history:
            ax2 = ax1.twinx()
            ax2.plot(history["lr"], label="Learning Rate", color='tab:green', linestyle="--", alpha=0.7)
            ax2.set_ylabel("Learning Rate", color='tab:green')
            ax2.tick_params(axis='y', labelcolor='tab:green')

        plt.title(f"Trial {trial.number} - Fold {fold_i + 1}")
        fig.tight_layout()
        fig.legend(loc="upper right", bbox_to_anchor=(1, 1), bbox_transform=ax1.transAxes)
        plt.show()

In [ ]:
#  Get best params
best_params = study.best_trial.params

#  Build model function that accepts params directly (not trial object)
def build_best_model(params):
    filter_size = 700
    padding_type = 'valid'
    num_filters = params['num_filters']  # or keep as params['num_filters'] if you want

    initializer = tf.keras.initializers.he_normal(seed=123)

    inputs = tf.keras.Input(shape=(700,))
    x = tf.keras.layers.Reshape((700, 1))(inputs)
    x = tf.keras.layers.Conv1D(filters=num_filters, kernel_size=filter_size,
                               padding=padding_type,
                               kernel_initializer=initializer,
                               kernel_regularizer=tf.keras.regularizers.l2(params['l2_reg']))(x)
    x = tf.keras.layers.Activation('elu')(x)
    print("Conv1D output shape:", x.shape)
    x = tf.keras.layers.Flatten()(x)

    for i in range(params['num_dense_layers']):
        units = params[f'dense_{i}_units']
        x = tf.keras.layers.Dense(units, kernel_initializer=initializer,
                                  kernel_regularizer=tf.keras.regularizers.l2(params['l2_reg']))(x)
        x = tf.keras.layers.Activation('elu')(x)
        if params['num_dense_layers'] > 1:
            x = tf.keras.layers.Dropout(rate=params['dropout_rate'])(x)

    output = tf.keras.layers.Dense(1, activation="linear", kernel_initializer=initializer)(x)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
                  loss="mse",
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

In [ ]:
# #OLD
# #  Cross-validation to estimate number of epochs
# kf = KFold(n_splits=5, shuffle=True, random_state=123)
# early_stopping_epochs = []

# for train_idx, val_idx in kf.split(X_train_f):
#     X_tr, X_v = X_train_f[train_idx], X_train_f[val_idx]
#     Y_tr, Y_v = Y_train.iloc[train_idx], Y_train.iloc[val_idx]
    
#     model = build_best_model(best_params)
#     es_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, min_delta=5e-3, restore_best_weights=True)
    
#     history = model.fit(X_tr, Y_tr,
#                         validation_data=(X_v, Y_v),
#                         epochs=500,
#                         batch_size=best_params['batch_size'],
#                         callbacks=[es_callback],
#                         verbose=0)
    
#     stopped_epoch = es_callback.stopped_epoch or len(history.history['loss'])
#     early_stopping_epochs.append(stopped_epoch)

# # 4. Estimate final number of epochs
# final_n_epochs = int(np.mean(early_stopping_epochs))
# print("the number of epochs for each fold:", early_stopping_epochs)
# print("Estimated number of epochs for final training:", final_n_epochs)

# for i in range(5): winsound.Beep(500, 500) 

In [ ]:
# Train final model on full training set with estimated epochs
final_model = build_best_model(best_params)
final_model.fit(X_train_f, Y_train,
                epochs=study.best_trial.user_attrs["avg_epochs"],
                batch_size=best_params['batch_size'],
                verbose=0)

# Evaluate on test set
test_loss, test_rmse = final_model.evaluate(X_test_f, Y_test, verbose=1)
print(f"RMSE on test set: {test_rmse:.4f}")

In [17]:
# Calcul des métriques
rmse = test_rmse
rpd = np.std(Y_test) / rmse
relative_error = rmse / np.mean(Y_test)

# Préparation de la nouvelle ligne
new_row = pd.DataFrame({
    "Modèle": [Modd],
    "Variable": [Var],
    "RMSE": [rmse],
    "RE": [relative_error],
    "RPD": [rpd]
})

# Chemin vers ton fichier CSV
csv_path = "C:/Users/Administrator/Desktop/PFE/otha/results/resultats_CNN.csv"

# Si le fichier existe déjà, on l'ouvre et on ajoute la nouvelle ligne
if os.path.exists(csv_path):
    existing_results = pd.read_csv(csv_path)
    updated_results = pd.concat([existing_results, new_row], ignore_index=True)
else:
    # Si le fichier n'existe pas encore, on crée un nouveau fichier avec juste cette ligne
    updated_results = new_row

# Sauvegarde
updated_results.to_csv(csv_path, index=False, sep=',')
updated_results

,Modèle,Variable,RMSE,RE,RPD
0,CNN-R_v1D,cp,1.187024,0.104192,3.918738
1,CNN-R_v1E,adf,3.389963,0.102010,2.945358
2,CNN-R_v1D,adf,3.940099,0.118565,2.534112
3,CNN-R_v1E,dmdcell,4.809555,0.090238,3.442970
4,CNN-R_v1D,dmdcell,5.119794,0.096058,3.234340
5,CNN-R_v1E,adl,2.946611,0.268297,2.635941
6,CNN-R_v1E,ndf,4.696548,0.089170,2.889557
7,CNN-R_v1D,ndf,3.938443,0.074776,3.445764
8,CNN-R_v1E,cf,2.939514,0.102038,3.113246
9,CNN-R_v1D,cf,2.857829,0.099203,3.202231


In [ ]:
# Save the model summary to a text file
Models = f"C:/Users/Administrator/Desktop/PFE/otha/models/Archs.txt"

# Open the file in append mode
with open(Models, "a") as file:
    # Write the header with the model name
    file.write(f" ************************************************************\n")
    file.write(f" ******************  Model: {Modd}__{Var}  ******************\n")
    file.write(f" ************************************************************\n")
    model_summary = []
    final_model.summary(print_fn=lambda x: model_summary.append(x))
    file.write("\n".join(model_summary))
    file.write("\n\n")  # Add some space between entries


In [ ]:
# Save best hyperparameters to a JSON file
params_path = f"C:/Users/Administrator/Desktop/PFE/otha/models/best parameters/best_params_{Modd}__{Var}.json"

with open(params_path, "a") as param_file:
    json.dump(best_params, param_file, indent=4)

<!-- ###  CNN-R_v1D -->

# CNN-R_v1E

In [ ]:
Modd = "CNN-R_v1D"

In [ ]:
def build_model(trial):
    # CNN-R_v1D logic
    filter_size = trial.suggest_int("filter_size", 5, 200, step=20)
    padding_type = 'same'
    num_filters = trial.suggest_int("num_filters", 1, 32)

    # Only 1 dense layer → no need for loop
    dense_units = trial.suggest_int("dense_0_units", 8, 128, step=4)

    learning_rate = trial.suggest_float("learning_rate", 1e-4, 0.03, log=True)
    l2_reg = trial.suggest_float("l2_reg", 0.0, 0.1, step=5e-4)
    batch_size = trial.suggest_int("batch_size", 32, 256, step=16)

    initializer = tf.keras.initializers.he_normal(seed=123)

    inputs = tf.keras.Input(shape=(700,))
    x = tf.keras.layers.Reshape((700, 1))(inputs)
    x = tf.keras.layers.Conv1D(filters=num_filters, kernel_size=filter_size,
                               padding=padding_type,
                               kernel_initializer=initializer,
                               kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x)
    # print("Conv1D output shape:", x.shape)
    x = tf.keras.layers.Activation('elu')(x)
    x = tf.keras.layers.Flatten()(x)

    # Single dense layer, no dropout
    x = tf.keras.layers.Dense(dense_units, kernel_initializer=initializer,
                              kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x)
    x = tf.keras.layers.Activation('elu')(x)

    output = tf.keras.layers.Dense(1, activation="linear", kernel_initializer=initializer)(x)
    model = tf.keras.Model(inputs=inputs, outputs=output)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss="mse",
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
    return model, batch_size

In [ ]:
# Start the Optuna study 
ii = datetime.now()
iii = str(ii.strftime("%Y-%m-%d_%Hh%M"))
print(" **********************************    Start time:", iii,"   ***********************************")

study = optuna.create_study(
    direction="minimize",
    study_name="cnn_hpo",
    # storage=f"sqlite:///cnn_study__{iii}.db",
    load_if_exists=True
)
study.optimize(objective, n_trials=100)

# Print the best trial
print("Best trial :")
print(f"  RMSE on Val: {study.best_value:.4f}")
print("  Best params:")
for k, v in study.best_trial.params.items():
    print(f"    {k}: {v}")

for i in range(5): winsound.Beep(500, 500)

In [ ]:
# Epochs and plots n stuff
print("Average epochs:", study.best_trial.user_attrs["avg_epochs"])
print("Epochs per fold:", study.best_trial.user_attrs["epochs_list"])

# Pick trial indices
start_idx = 0
middle_idx = len(study.trials) // 2
end_idx = len(study.trials) -1
chosen_idxs = [start_idx, middle_idx, end_idx]

for idx in chosen_idxs:
    trial = study.trials[idx]
    histories = trial.user_attrs.get("fold_histories", [])

    for fold_i, history in enumerate(histories):
        fig, ax1 = plt.subplots(figsize=(10, 4))

        # Plot Losses on left axis
        ax1.plot(history["loss"], label="Training Loss", color='tab:blue')
        ax1.plot(history["val_loss"], label="Validation Loss", color='tab:orange')
        ax1.set_xlabel("Epoch")
        ax1.set_ylabel("Loss", color='black')
        ax1.tick_params(axis='y', labelcolor='black')
        ax1.grid(True)

        # Plot Learning Rate on right axis if available
        if "lr" in history:
            ax2 = ax1.twinx()
            ax2.plot(history["lr"], label="Learning Rate", color='tab:green', linestyle="--", alpha=0.7)
            ax2.set_ylabel("Learning Rate", color='tab:green')
            ax2.tick_params(axis='y', labelcolor='tab:green')

        plt.title(f"Trial {trial.number} - Fold {fold_i + 1}")
        fig.tight_layout()
        fig.legend(loc="upper right", bbox_to_anchor=(1, 1), bbox_transform=ax1.transAxes)
        plt.show()

In [ ]:
best_params = study.best_trial.params

def build_best_model(params):
    filter_size = params['filter_size']
    padding_type = 'same'  # This is what they used in v1D
    num_filters = params['num_filters']

    initializer = tf.keras.initializers.he_normal(seed=123)

    inputs = tf.keras.Input(shape=(700,))
    x = tf.keras.layers.Reshape((700, 1))(inputs)
    x = tf.keras.layers.Conv1D(
        filters=num_filters,
        kernel_size=filter_size,
        padding=padding_type,
        kernel_initializer=initializer,
        kernel_regularizer=tf.keras.regularizers.l2(params['l2_reg'])
    )(x)
    x = tf.keras.layers.Activation('elu')(x)
    x = tf.keras.layers.Flatten()(x)

    # Only one dense layer, no dropout
    units = params['dense_0_units']
    x = tf.keras.layers.Dense(
        units,
        kernel_initializer=initializer,
        kernel_regularizer=tf.keras.regularizers.l2(params['l2_reg'])
    )(x)
    x = tf.keras.layers.Activation('elu')(x)

    output = tf.keras.layers.Dense(1, activation="linear", kernel_initializer=initializer)(x)
    model = tf.keras.Model(inputs=inputs, outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )
    return model


In [ ]:
# #  Cross-validation to estimate number of epochs
# kf = KFold(n_splits=5, shuffle=True, random_state=123)
# early_stopping_epochs = []

# for train_idx, val_idx in kf.split(X_train_f):
#     X_tr, X_val = X_train_f[train_idx], X_train_f[val_idx]
#     Y_tr, Y_val = Y_train.iloc[train_idx], Y_train.iloc[val_idx]
    
#     model = build_best_model(best_params)
#     es_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, min_delta=5e-3, restore_best_weights=True)
    
#     history = model.fit(X_tr, Y_tr,
#                         validation_data=(X_val, Y_val),
#                         epochs=400,
#                         batch_size=best_params['batch_size'],
#                         callbacks=[es_callback],
#                         verbose=0)
    
#     stopped_epoch = es_callback.stopped_epoch or len(history.history['loss'])
#     early_stopping_epochs.append(stopped_epoch)

# #  Estimate final number of epochs
# final_n_epochs = int(np.mean(early_stopping_epochs))
# print("the number of epochs for each fold:", early_stopping_epochs)
# print("Estimated number of epochs for final training:", final_n_epochs)

# for i in range(5): winsound.Beep(500, 500)


In [ ]:
# Train final model on full training set with estimated epochs
final_model = build_best_model(best_params)
final_model.fit(X_train_f, Y_train,
                epochs=study.best_trial.user_attrs["avg_epochs"],
                batch_size=best_params['batch_size'],
                verbose=0)

# 6. Evaluate on test set
test_loss, test_rmse = final_model.evaluate(X_test_f, Y_test, verbose=1)
print(f"RMSE on test set: {test_rmse:.4f}")

<!-- # Enregistrer les Résultats -->

In [ ]:
# Calcul des métriques
rmse = test_rmse
rpd = np.std(Y_test) / rmse
relative_error = rmse / np.mean(Y_test)

# Préparation de la nouvelle ligne
new_row = pd.DataFrame({
    "Modèle": [Modd],
    "Variable": [Var],
    "RMSE": [rmse],
    "RE": [relative_error],
    "RPD": [rpd]
})

# Chemin vers ton fichier CSV
csv_path = "C:/Users/Administrator/Desktop/PFE/otha/results/resultats_CNN.csv"

# Si le fichier existe déjà, on l'ouvre et on ajoute la nouvelle ligne
if os.path.exists(csv_path):
    existing_results = pd.read_csv(csv_path)
    updated_results = pd.concat([existing_results, new_row], ignore_index=True)
else:
    # Si le fichier n'existe pas encore, on crée un nouveau fichier avec juste cette ligne
    updated_results = new_row

# Sauvegarde
updated_results.to_csv(csv_path, index=False, sep=',')
updated_results

In [ ]:
# Save the model summary to a text file
Models = f"C:/Users/Administrator/Desktop/PFE/otha/models/archs.txt"

# Open the file in append mode
with open(Models, "a") as file:
    # Write the header with the model name
    file.write(f" ************************************************************\n")
    file.write(f" ******************  Model: {Modd}__{Var}  ******************\n")
    file.write(f" ************************************************************\n")
    model_summary = []
    final_model.summary(print_fn=lambda x: model_summary.append(x))
    file.write("\n".join(model_summary))
    file.write("\n\n")  # Add some space between entries


In [ ]:
# Save best hyperparameters to a JSON file
params_path = f"C:/Users/Administrator/Desktop/PFE/otha/models/best parameters/best_params_{Modd}__{Var}.json"

with open(params_path, "a") as param_file:
    json.dump(best_params, param_file, indent=4)

In [ ]:
# csv_path = "C:/Users/Administrator/Desktop/PFE/otha/results/resultats_CNN.csv"
# dff = pd.read_csv(csv_path)
# # Remove duplicate rows (keeping only the first occurrence)
# df_unique = dff.drop_duplicates(subset=['Modèle', 'Variable'], keep='last')
# # Save the cleaned CSV
# df_unique.to_csv(csv_path, index=False, sep=',')

In [ ]:
# ee = pd.read_csv(csv_path)

# ee.rename(columns={"RMSEP": "RMSE"}, inplace=True)
# ee.sort_values(by="Modèle", ascending=True, inplace=True)

# df_model_E = ee[ee['Modèle'] == 'CNN-R_v1E']
# df_model_D = ee[ee['Modèle'] == 'CNN-R_v1D']

# df_model_E.to_csv("C:/Users/Administrator/Desktop/PFE/otha/results/results_CNN-R_v1E.csv", index=False)
# df_model_D.to_csv("C:/Users/Administrator/Desktop/PFE/otha/results/results_CNN-R_v1D.csv", index=False)